In [2]:
import requests
import sys
import pandas as pd
from pandas import json_normalize
import json

# Spider Project's API Server
url = 'http://127.0.0.1:8080'
# Please prepend with the full path
filename = 'C:\\py\\AD_Aksai_S.1054.sprj'
reportName = 'C:\\Users\\DjumagalievE\\Documents\\Spider Project\\My report\\dsmDays.001.sdoc'

# return docHandle.key
project = requests.post(url, json={'command': 'openFile', 'fileName': filename, 'sessId': ''})
print('\nProject opened.') if project.status_code == 200 else sys.exit(0)
docHandleProject = project.json()['docHandle']

# execute Script
project = requests.post(url, json={"command":"executeScript","docHandle": docHandleProject,'scriptLibName': 'needDSMDays', 'sessId': ''})
print('\nScript executed.') if project.status_code == 200 else sys.exit(0)

# return docHandle.key
report = requests.post(url, json={'command': 'openFile', 'fileName': reportName, 'sessId': ''})
print('\nReport opened.') if report.status_code == 200 else sys.exit(0)
docHandleReport = report.json()['docHandle']

# return tableHandle.key
report = requests.post(url, json={'command':'getTableHandle','docHandle': docHandleReport,'sessId':''})
print('\nTable handle obtained.') if report.status_code == 200 else sys.exit(0)
tableHandle = report.json()['tableHandle']

# return table
report = requests.post(url, json={'command':'getTable','tableHandle': str(tableHandle),'sessId':''})
print('\nTable obtained.') if report.status_code == 200 else sys.exit(0)
data = report.json()['array']

data = str(data).replace("'",'"')
data = json.loads(data)

data = json_normalize(data)

# обработка NaN значений
data = data.fillna(0)
# замена типов данных
for i in data:
    if data[i].dtypes == 'float64':
        data[i] = data[i].astype('int64')
    else:
        continue
        
# Фреймы с данными за первые 20 дней месяца и последние 20 дней месяца
data_f20_days = data.iloc[:, 5:25]
data_l20_days = data.iloc[:, -20:]

# Серии со средними значениями по каждой строке
data_f20_days = data_f20_days.mean(axis=1)
data_l20_days = data_l20_days.mean(axis=1)

# конкатенация data_f20_days и data_l20_days по максимальному значению из 2 средних
data_max_mean = pd.concat([data_f20_days, data_l20_days], axis=1).max(axis=1)

# слияние в итоговый DataFrame (столбец ParName с типами техники + максимальное среднее)
fdata = data.iloc[:, 2:3].merge(data_max_mean.to_frame(), left_index=True, right_index=True)
fdata = fdata.rename(columns={0: 'MaxMean'})
fdata['MaxMean'] = fdata['MaxMean'].astype('int64')
fdata


Project opened.

Script executed.

Report opened.

Table handle obtained.

Table obtained.


,ParName,MaxMean
0,Аксай_5этап,0
1,Автобетоносмеситель Количество,1
2,Автогрейдер Количество,4
3,Автогудронатор Количество,0
4,Автокран грузоподъемностью до 25 т Количество,1
5,Автомобиль бортовой Количество,1
6,Автомобиль бортовой с КМУ Количество,0
7,Автопланировщик Количество,0
8,"Автосамосвал, грузоподъемность от 22 до 27 т К...",134
9,Асфальтоперегружатель Количество,0
